In [72]:
!pip install beautifulsoup4
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from tokenizers import Tokenizer  
from tokenizers import normalizers 
import nltk
from string import punctuation
import re
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
url = 'https://www.cnn.com/search/?q=apple'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

articles = soup.find_all('div', {'class': 'cnn-search__result--article'})

for article in articles:
    article_url = article.find('a')['href']
    article_title = article.find('h3').text.strip()
    article_date = article.find('div', {'class': 'cnn-search__result-publish-date'}).text.strip()

    print(article_url, article_title, article_date)

In [75]:
company_list = ['Goldman Sachs', 'Wells Fargo & Co', 'Google', 'Facebook', 'Apple']

In [76]:
link_dict = {}
for i in range(len(company_list)):
  for j in range(30):
    links = []
    url = 'https://www.bbc.co.uk/search?q={}&d=HOMEPAGE_GNL&page={}'.format(company_list[i],j+1)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    lists = soup.findAll('ul', attrs={'role': 'list'})
    for ul in lists:
      for link in ul.findAll('a', attrs={'class':'e1f5wbog1'}):
        url = link.get('href')
        if (not re.search("(.*programmes.*)", url)):
          links.append(link.get('href'))
    if (company_list[i] not in link_dict):
      link_dict[company_list[i]] = links
    else:
      link_dict[company_list[i]]+=links


for company in link_dict:
  print(len(link_dict[company]))




273
222
237
223
179


In [77]:
print(link_dict)

{'Goldman Sachs': ['https://www.bbc.co.uk/news/business-64906691', 'https://www.bbc.co.uk/news/uk-england-kent-63821885', 'https://www.bbc.co.uk/news/business-64234129', 'https://www.bbc.co.uk/news/business-64004299', 'https://www.bbc.co.uk/news/business-63012000', 'https://www.bbc.co.uk/news/business-62885047', 'https://www.bbc.co.uk/news/business-60854153', 'http://www.bbc.co.uk/news/business-36967923', 'https://www.bbc.co.uk/news/business-58054983', 'https://www.bbc.co.uk/news/world-latin-america-40101931', 'http://news.bbc.co.uk/1/hi/business/8646861.stm', 'http://news.bbc.co.uk/1/hi/business/8646931.stm', 'https://www.bbc.co.uk/news/business-35319938', 'https://www.bbc.co.uk/news/business-54876529', 'https://www.bbc.co.uk/news/business-36661181', 'https://www.bbc.co.uk/news/business-39815738', 'https://www.bbc.co.uk/news/world-africa-37657633', 'http://news.bbc.co.uk/today/hi/today/newsid_8631000/8631359.stm', 'https://www.bbc.co.uk/news/business-24425808', 'https://www.bbc.co.uk/

In [78]:
!pip install transformers datasets
!pip install torch
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [79]:
from transformers import pipeline
!pip3 install emoji==0.6.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
def get_sentimental_analysis(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  # Extract the text content of the article
  text = ""
  for paragraph in soup.find_all("p"):
      text += paragraph.text

  text = text.lower()
  text = nltk.word_tokenize(text)
  stop_words = set(nltk.corpus.stopwords.words('english'))
  punct = list(punctuation)
  punct.append(' ')
  text = [w for w in text if not w.lower() in stop_words]
  text = [word for word in text if word not in punct]
  tokenized = ''
  for word in text:
      tokenized += '{} '.format(word)

  # Analyze the sentiment of the article using TextBlob
  tokenized = tokenized[:100]

  specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
  res = specific_model(tokenized)
  # print(res)
  # blob = TextBlob(tokenized)
  # sentiment = blob.sentiment.polarity
  # subjectivity = blob.sentiment.subjectivity
  # print(type(res))
  # print(len(res))
  # print(res)
  for x in res:
    label = x["label"] 
    score = x["score"]
  # print(label, " + ", score)
  timestamp = ''
  timeEl = soup.find('time')
  if (timeEl):
    try:
        timestamp = timeEl['datetime']
    except:
        return (label, score,timestamp,text)
    
  return (label, score,timestamp,text)

# get_sentimental_analysis('https://www.bbc.co.uk/programmes/b00sg1nj')

In [81]:
!pip install tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
dataset2 = {
    'company_name':[],
    'timestamp':[],
    'url':[],
    'label':[],
    'score':[],
    'text':[]
}
for company in tqdm(company_list, desc = "Company List: "):
  url_list = link_dict[company]
  for url in url_list:
    (label, score, timestamp, text) = get_sentimental_analysis(url)
    # get_sentimental_analysis(url)
    dataset2['company_name'].append(company)
    dataset2['timestamp'].append(timestamp)
    dataset2['url'].append(url)
    dataset2['label'].append(label)
    dataset2['score'].append(score)
    dataset2['text'].append(text)

df2 = pd.DataFrame(dataset2)
    

Company List: 100%|██████████| 5/5 [1:07:51<00:00, 814.36s/it]


In [86]:
df2

,company_name,timestamp,url,label,score,text
0,Goldman Sachs,2023-03-09T19:28:44.000Z,https://www.bbc.co.uk/news/business-64906691,NEG,0.898829,"[former, head, goldman, sachs, malaysia, sent,..."
1,Goldman Sachs,2022-12-02T06:39:38.000Z,https://www.bbc.co.uk/news/uk-england-kent-638...,NEG,0.927286,"[woman, conned, £120,000, fraudster, posed, go..."
2,Goldman Sachs,2023-01-11T11:06:18.000Z,https://www.bbc.co.uk/news/business-64234129,NEU,0.649682,"[investment, giant, goldman, sachs, begun, mas..."
3,Goldman Sachs,2022-12-16T17:57:54.000Z,https://www.bbc.co.uk/news/business-64004299,NEU,0.654611,"[goldman, sachs, planning, deep, job, cuts, gr..."
4,Goldman Sachs,2022-09-23T17:47:11.000Z,https://www.bbc.co.uk/news/business-63012000,NEG,0.932344,"[women, goldman, sachs, reported, 75, incident..."
...,...,...,...,...,...,...
1129,Apple,2019-06-07T16:00:35.000Z,https://www.bbc.co.uk/news/technology-48555156,POS,0.837496,"[every, apple, event, every, detail, new, mac,..."
1130,Apple,2019-04-29T11:00:47.000Z,https://www.bbc.co.uk/news/technology-48092151,NEU,0.638554,"[apple, defended, decision, remove, number, pa..."
1131,Apple,2019-04-23T14:54:08.000Z,https://www.bbc.co.uk/news/technology-48022890,NEG,0.756685,"[student, suing, apple, inc, 1bn, £0.77bn, cla..."
1132,Apple,2021-09-03T15:16:05.000Z,https://www.bbc.co.uk/news/technology-58433647,NEG,0.816005,"[apple, delayed, plans, roll, detection, techn..."


In [49]:
dataset = {
    'company_name':[],
    'timestamp':[],
    'url':[],
    'polarity':[],
    'subjectivity':[],
    'text':[]
}
for company in company_list:
  url_list = link_dict[company]
  for url in url_list:
    # (sentiment, subjectivity, timestamp, text) = get_sentimental_analysis(url)
    get_sentimental_analysis(url)
    # dataset['company_name'].append(company)
    # dataset['timestamp'].append(timestamp)
    # dataset['url'].append(url)
    # dataset['polarity'].append(sentiment)
    # dataset['subjectivity'].append(subjectivity)
    # dataset['text'].append(text)

df = pd.DataFrame(dataset)
    

[{'label': 'NEG', 'score': 0.898829460144043}]
[{'label': 'NEG', 'score': 0.9272856712341309}]
[{'label': 'NEU', 'score': 0.6496817469596863}]
[{'label': 'NEU', 'score': 0.6546114683151245}]
[{'label': 'NEG', 'score': 0.9323436617851257}]
[{'label': 'NEG', 'score': 0.887285590171814}]
[{'label': 'NEU', 'score': 0.8581705093383789}]
[{'label': 'NEG', 'score': 0.8592300415039062}]
[{'label': 'NEU', 'score': 0.7996866106987}]
[{'label': 'NEU', 'score': 0.4978660047054291}]
[{'label': 'NEU', 'score': 0.9468315839767456}]
[{'label': 'NEU', 'score': 0.7295109033584595}]
[{'label': 'NEU', 'score': 0.6035552620887756}]
[{'label': 'POS', 'score': 0.6093858480453491}]
[{'label': 'NEU', 'score': 0.9640107750892639}]
[{'label': 'NEU', 'score': 0.945663571357727}]
[{'label': 'NEU', 'score': 0.8221437931060791}]
[{'label': 'NEU', 'score': 0.9602001309394836}]
[{'label': 'NEU', 'score': 0.9368889927864075}]
[{'label': 'NEU', 'score': 0.9253710508346558}]
[{'label': 'NEU', 'score': 0.9346802830696106}

KeyboardInterrupt: ignored

In [8]:
df

,company_name,timestamp,url,polarity,subjectivity,text
0,Goldman Sachs,2023-03-09T19:28:44.000Z,https://www.bbc.co.uk/news/business-64906691,0.088333,0.360556,"[former, head, goldman, sachs, malaysia, sent,..."
1,Goldman Sachs,2023-01-11T11:06:18.000Z,https://www.bbc.co.uk/news/business-64234129,0.088605,0.437919,"[investment, giant, goldman, sachs, begun, mas..."
2,Goldman Sachs,2022-12-02T06:39:38.000Z,https://www.bbc.co.uk/news/uk-england-kent-638...,0.083408,0.465179,"[woman, conned, £120,000, fraudster, posed, go..."
3,Goldman Sachs,2022-12-16T17:57:54.000Z,https://www.bbc.co.uk/news/business-64004299,0.109425,0.431151,"[goldman, sachs, planning, deep, job, cuts, gr..."
4,Goldman Sachs,2022-09-23T17:47:11.000Z,https://www.bbc.co.uk/news/business-63012000,0.079449,0.330386,"[women, goldman, sachs, reported, 75, incident..."
...,...,...,...,...,...,...
1130,Apple,2019-06-07T16:00:35.000Z,https://www.bbc.co.uk/news/technology-48555156,0.192316,0.478797,"[every, apple, event, every, detail, new, mac,..."
1131,Apple,2019-04-29T11:00:47.000Z,https://www.bbc.co.uk/news/technology-48092151,0.193636,0.471288,"[apple, defended, decision, remove, number, pa..."
1132,Apple,2021-09-03T15:16:05.000Z,https://www.bbc.co.uk/news/technology-58433647,0.150926,0.490432,"[apple, delayed, plans, roll, detection, techn..."
1133,Apple,2019-04-23T14:54:08.000Z,https://www.bbc.co.uk/news/technology-48022890,0.137446,0.411526,"[student, suing, apple, inc, 1bn, £0.77bn, cla..."
